In [1]:
from modules.building_data import get_tile, osmbuildings_request, receive_building_data, polyarea, get_tile, tile_bbox

In [2]:
from modules.building_data import get_all_tiles, get_all_tile_jsons, get_tiles

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import mpmath as mp
import requests
import pandas as pd
import warnings
from shapely.geometry import Polygon
from modules.polygon_intersections import PolygonClipper
from ast import literal_eval

In [5]:
paris_df = pd.read_csv("../data/processed_data/Paris/Paris.csv")

In [6]:
paris_df.head()

,LST,ele,LST_diff,ele_diff,ul_corner,ll_corner,lr_corner,ur_corner,bb
0,28.489990,45.577310,-2.315979,-28.184334,"[[2.2633263464873545, 48.81321032181281]]","[[2.2633263464873545, 48.813839888049166]]","[[2.2639559127237074, 48.813839888049166]]","[[2.2639559127237074, 48.81321032181281]]","[[2.2633263464873545, 48.81321032181281], [2.2..."
1,28.549988,45.577430,-2.255981,-28.184212,"[[2.2639559127237074, 48.81321032181281]]","[[2.2639559127237074, 48.813839888049166]]","[[2.2645854789600603, 48.813839888049166]]","[[2.2645854789600603, 48.81321032181281]]","[[2.2639559127237074, 48.81321032181281], [2.2..."
2,28.450012,46.947140,-2.355957,-26.814503,"[[2.2645854789600603, 48.81321032181281]]","[[2.2645854789600603, 48.813839888049166]]","[[2.2652150451964133, 48.813839888049166]]","[[2.2652150451964133, 48.81321032181281]]","[[2.2645854789600603, 48.81321032181281], [2.2..."
3,29.230011,48.411680,-1.575958,-25.349964,"[[2.2652150451964133, 48.81321032181281]]","[[2.2652150451964133, 48.813839888049166]]","[[2.265844611432766, 48.813839888049166]]","[[2.265844611432766, 48.81321032181281]]","[[2.2652150451964133, 48.81321032181281], [2.2..."
4,27.889984,48.643574,-2.915985,-25.118069,"[[2.265844611432766, 48.81321032181281]]","[[2.265844611432766, 48.813839888049166]]","[[2.266474177669119, 48.813839888049166]]","[[2.266474177669119, 48.81321032181281]]","[[2.265844611432766, 48.81321032181281], [2.26..."


In [7]:
paris_df['test'] = paris_df['bb'].copy()

In [8]:
paris_df['test'] = paris_df.test.apply(literal_eval)

In [9]:
type(paris_df.test[0])

list

In [ ]:
# paris_df.bb.apply(lambda x: x.)

In [ ]:
# paris_df['test'] = paris_df['test'].replace('[]', '')

In [ ]:
# paris_df.bb.apply(lambda x: x.)

In [10]:
clip = PolygonClipper(warn_if_empty=False)

In [11]:
def get_building_clipped_area(building, clip, pixel):
    building_coords = building[1]
    clipped = clip(pixel, building_coords)
    pgon = Polygon(clipped)
    return pgon.area

In [12]:
def get_building_coverage(buildings, clip, pixel):
    separate_areas = []
    for building in buildings:
        try:
            separate_areas.append(get_building_clipped_area(building, clip, pixel))
        except:
            continue
    total_area = np.sum(separate_areas)
    pixel_poly = Polygon(pixel)
    pixel_area = pixel_poly.area
    prop_area_built = total_area/pixel_area
    return prop_area_built

In [13]:
paris_coords = {'upper_left': [48.813898, 2.264216],
                'lower_right': [48.900502, 2.42172]}

In [14]:
paris_tiles = get_tiles(paris_coords)
paris_x_tiles, paris_y_tiles = get_all_tiles(paris_coords, paris_tiles)
paris_jsons = get_all_tile_jsons(paris_x_tiles, paris_y_tiles)
paris_buildings = receive_building_data(paris_jsons)

URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11279.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11280.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11281.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11282.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11283.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11284.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11285.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11286.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11287.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11288.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16590/11289.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16591/11279.json
URL: https://data.osmbuildings.org/0.2/anonymous/tile/15/16591/11280.json
URL: https://data.osmbuildings.org/0.2

In [15]:
paris_buildings[0]

[3,
 [[2.264284, 48.80854],
  [2.264271, 48.808532],
  [2.264219, 48.808564],
  [2.264237, 48.808584],
  [2.264256, 48.808572],
  [2.264245, 48.808564],
  [2.264284, 48.80854]]]

In [16]:
single_pixel = paris_df['test'][0]

In [17]:
single_pixel

[[2.2633263464873545, 48.81321032181281],
 [2.2633263464873545, 48.813839888049166],
 [2.2639559127237074, 48.813839888049166],
 [2.2639559127237074, 48.81321032181281]]

In [29]:
building_density = get_building_coverage(paris_buildings, clip, single_pixel)

CPU times: user 2.92 s, sys: 0 ns, total: 2.92 s
Wall time: 2.91 s


In [30]:
building_density

0.027406419466892006

In [20]:
test_set = paris_df['test'][:10]

In [25]:
building_densities = [get_building_coverage(paris_buildings, clip, pixel) for pixel in test_set]

In [26]:
building_densities

[0.027406419466892006,
 0.00861925398265395,
 0.06005162768672451,
 0.004278246729816333,
 0.01312244811213052,
 0.017363859450119708,
 0.03459807158376528,
 0.010080973420051632,
 0.047053275099517414,
 0.05453027720990245]

In [39]:
# paris_df['building_density'] = paris_df.test.apply(lambda x : get_building_coverage(paris_buildings, clip, x))

In [ ]:
# paris_df['building_density'].value_counts()